This data was obtained from the client’s inventory database using Microsoft SQL Server. A two-year history of the products that have been issued from the distribution center to the facilities was obtained.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta
from datetime import datetime


df = pd.read_csv (r"F:\Shr_Priv\Materials_Mgt_DataTeam\Laura Eshee\Demand Forcasting\Demand Forecasting Query Sample.csv", index_col=0, low_memory=False, skipinitialspace=True)

df.reset_index(inplace=True, drop=True)

print(df.head())
print(df.info())


   Company   Item                     Description  Qty STOCK_UOM Trans_UOM  \
0      240  59417  UNDERPAD INCONT 24X17IN         300      EA        CS     
1      300  59417  UNDERPAD INCONT 24X17IN         300      EA        CS     
2      300  59417  UNDERPAD INCONT 24X17IN         300      EA        CS     
3      300  59417  UNDERPAD INCONT 24X17IN         900      EA        CS     
4      300  59417  UNDERPAD INCONT 24X17IN         600      EA        CS     

   UNIT_COST  Ext Amount TRANS_DATE Item Type  ... ALT_UOM_CONV_03  \
0     0.0601       18.03   5/4/2020         I  ...               0   
1     0.0601       18.03   5/5/2020         I  ...               0   
2     0.0601       18.03   5/5/2020         I  ...               0   
3     0.0601       54.09  5/27/2020         I  ...               0   
4     0.0601       36.06  6/18/2020         I  ...               0   

   ALT_UOM_CONV_04 ALT_UOM_CONV_05  ALT_UOM_CONV_06  BUY_FL_01  BUY_FL_02  \
0                0               

The date column was imported as an object data type. It will now be converted to a datetime data type.

In [2]:
df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109157 entries, 0 to 109156
Data columns (total 44 columns):
Company            109157 non-null int64
Item               109157 non-null int64
Description        109157 non-null object
Qty                109157 non-null int64
STOCK_UOM          109157 non-null object
Trans_UOM          109157 non-null object
UNIT_COST          109157 non-null float64
Ext Amount         109157 non-null float64
TRANS_DATE         109157 non-null datetime64[ns]
Item Type          109157 non-null object
Sys                109157 non-null object
Document           109157 non-null int64
Doc Type           109157 non-null object
Line Nbr           109157 non-null int64
Req Nbr            109157 non-null int64
From Location      109157 non-null int64
Req Location       109157 non-null object
TRACKING_FL_01     109157 non-null object
TRACKING_FL_02     0 non-null float64
TRACKING_FL_03     0 non-null float64
TRACKING_FL_04     0 non-null float64
TRACKING_FL_05  

The client requested that the unit of measure (UOM) used for forecasting be the tracked UOM. The tracked UOM is denoted in the data by an X in one of the BUY_FL_ columns. The code below converts the transaction UOM to the tracked UOM.

In [3]:
def conv(df):
    if (df['BUY_FL_01'] == 'X'):
        return df['ALT_UOM_CONV_01'].apply(pd.to_numeric, errors='coerce')
    elif (df['BUY_FL_02'] == 'X'):
        return df['ALT_UOM_CONV_02'].apply(pd.to_numeric, errors='coerce')
    elif (df['BUY_FL_03'] == 'X'): 
        return df['ALT_UOM_CONV_03'].apply(pd.to_numeric, errors='coerce')
    elif (df['BUY_FL_04'] == 'X'):
        return df['ALT_UOM_CONV_04'].apply(pd.to_numeric, errors='coerce')         
    elif (df['BUY_FL_05'] == 'X'): 
        return df['ALT_UOM_CONV_05'].apply(pd.to_numeric, errors='coerce')
    elif (df['BUY_FL_06'] == 'X'):
        return df['ALT_UOM_CONV_06'].apply(pd.to_numeric, errors='coerce')
    else:
        return 1
                    
df['Track_UOM_Conv'] = df.apply(conv, axis=1)
                        
print(df['Track_UOM_Conv'])

0         1
1         1
2         1
3         1
4         1
         ..
109152    1
109153    1
109154    1
109155    1
109156    1
Name: Track_UOM_Conv, Length: 109157, dtype: int64


Using the conversion amount calculated above, the transaction quantity is converted to the tracked UOM quantity by multiplying Qty by Track_UOM_Conv.

In [4]:
df['conv_qty'] = (df.Track_UOM_Conv * df.Qty).astype(int)

print(df.conv_qty)
print (df.info())

0         300
1         300
2         300
3         900
4         600
         ... 
109152    200
109153    200
109154    400
109155    200
109156    400
Name: conv_qty, Length: 109157, dtype: int32
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109157 entries, 0 to 109156
Data columns (total 46 columns):
Company            109157 non-null int64
Item               109157 non-null int64
Description        109157 non-null object
Qty                109157 non-null int64
STOCK_UOM          109157 non-null object
Trans_UOM          109157 non-null object
UNIT_COST          109157 non-null float64
Ext Amount         109157 non-null float64
TRANS_DATE         109157 non-null datetime64[ns]
Item Type          109157 non-null object
Sys                109157 non-null object
Document           109157 non-null int64
Doc Type           109157 non-null object
Line Nbr           109157 non-null int64
Req Nbr            109157 non-null int64
From Location      109157 non-null int64
Req Location   

The data wrangling is complete, and the wrangled data frame is saved into a csv file.

In [5]:
df.to_csv("F:\Shr_Priv\Materials_Mgt_DataTeam\Laura Eshee\Demand Forcasting\Output Files\Demand Forecasting Data Wrangling Sample.csv")
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109157 entries, 0 to 109156
Data columns (total 46 columns):
Company            109157 non-null int64
Item               109157 non-null int64
Description        109157 non-null object
Qty                109157 non-null int64
STOCK_UOM          109157 non-null object
Trans_UOM          109157 non-null object
UNIT_COST          109157 non-null float64
Ext Amount         109157 non-null float64
TRANS_DATE         109157 non-null datetime64[ns]
Item Type          109157 non-null object
Sys                109157 non-null object
Document           109157 non-null int64
Doc Type           109157 non-null object
Line Nbr           109157 non-null int64
Req Nbr            109157 non-null int64
From Location      109157 non-null int64
Req Location       109157 non-null object
TRACKING_FL_01     109157 non-null object
TRACKING_FL_02     0 non-null float64
TRACKING_FL_03     0 non-null float64
TRACKING_FL_04     0 non-null float64
TRACKING_FL_05  

The data wrangling for this data set was fairly simple. First, the date column data type was converted from object to datetime. Then, the transaction UOM was converted to the tracked UOM and the tracked quantity was computed. The data set was then saved to a csv file. It is now ready for the next step.